### Задачи к Лекции 4

__Исходные данные__ 

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]


df = pd.read_csv("/content/drive/MyDrive/ML/ml2021p/data/mlbootcamp5_train.csv", 
                 sep=";", 
                 index_col="id")
# Делаем one-hot кодирование
chol = pd.get_dummies(df["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df["gluc"], prefix="gluc")
df = pd.concat([df, chol, gluc], axis=1)

# Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3,gender_bin
id,,,,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,1,0,0,1,0,0,1
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,0,0,1,1,0,0,0
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,0,0,1,1,0,0,0
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,1,0,0,1,0,0,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,1,0,0,1,0,0,0


In [32]:
num_cols = ['age',
            'height',
            'weight',
            'ap_hi',
            'ap_lo']

## Задачи

__1. Хоть в sklearn и присутствует реализация метода k-ближайших соседей, я же предлагаю попробовать вам написать его самостоятельно.__

* __создать классификатор используя только pandas, numpy и scipy. Гиперпараметром данного классификатора должно быть число ближайших соседей. (Необязательно) можно добавить метрику расстояния и выбор весов.__
* __С помощью кросс-валидации найти оптимальное количество ближайших соседей и (необязательно) набор признаков.__

Алгоритм работы классификатора:
 1. Для заданного прецедент  $\vec{x}$ мы считаем расстояние до всех прецедентов в обучающей выборке.
 2. Сортируем прецеденты по расстоянию до $\vec{x}$.
 3. Отбираем $k$ минимальных значений
 4. Устраиваем голосование между отобранными прецедент.

In [76]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

In [35]:
class KNN():
    
    def __init__(self, k=1):
        self.k = k
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
    def predict(self, X):
        dists = self.compute_distances(X)
        
        num_test = dists.shape[0]
        pred = np.zeros(num_test, np.bool)
        for i in tqdm(range(num_test)):
            ind = dists[i].argsort()[:self.k]
            pred[i] = bool(np.median(self.y_train[ind]))
            
        return pred
    
    def compute_distances(self, X):
        num_train = self.X_train.shape[0]
        num_test = X.shape[0]
        dists = np.zeros((num_test, num_train), np.float32)
        for i_test in tqdm(range(num_test)):
            dists[i_test] = np.sum(np.abs(X[i_test] - self.X_train), axis=1)

        return dists

In [36]:
X = df.drop(['cardio'], axis=1)
y = df['cardio']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

In [37]:
scaler = StandardScaler()

scaler.fit(X_train[num_cols].values.reshape(-1, 5))

X_train[num_cols] = scaler.transform(X_train[num_cols].values.reshape(-1, 5))
X_test[num_cols] = scaler.transform(X_test[num_cols].values.reshape(-1, 5))

In [ ]:
def MyKNNGridSearchCV(params, X, y, cv):

    X = X.values
    y = y.values

    scores = []

    kf = KFold(n_splits=cv, random_state=42, shuffle=False)

    for param in params:

        for train_index, test_index in kf.split(X):

            data_train, data_test, data_train_target, data_test_target = X[train_index], X[test_index], y[train_index], y[test_index]

        model = KNN(param)
        model.fit(data_train, data_train_target)
        pred = model.predict(data_test)

        scores.append(accuracy_score(data_test_target, pred))

    return scores

In [ ]:
model_CV = MyKNNGridSearchCV([2, 3, 4, 5, 6], X_train, y_train, 5)

100%|██████████| 9800/9800 [00:38<00:00, 252.53it/s]


In [ ]:
best_score = np.max(model_CV)
best_score, model_CV

(0.6639795918367347,
 [0.6187755102040816,
  0.6455102040816326,
  0.6532653061224489,
  0.6639795918367347,
  0.6618367346938776])

In [ ]:
X = df.drop(['cardio'], axis=1)
y = df['cardio'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

scaler = StandardScaler()

scaler.fit(X_train[num_cols].values.reshape(-1, 5))

X_train[num_cols] = scaler.transform(X_train[num_cols].values.reshape(-1, 5))
X_test[num_cols] = scaler.transform(X_test[num_cols].values.reshape(-1, 5))

X_train = X_train.values
X_test = X_test.values

model = KNN(5)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

100%|██████████| 21000/21000 [01:45<00:00, 198.93it/s]


0.6591904761904762

**Комментарии:** Без масштабирования признаков, score был больше

**2. Определить какой из трех классификаторов (kNN, наивный Байес, решающее дерево) лучший в каждой метрике по отдельности: accuracy, F1-мера, ROC AUC, функция потерь. Использовать набор признаков: 'age', 'weight', 'height', 'ap_lo', 'ap_hi'.**

**(Необязательно) Найти оптимальный набор признаков.**

In [103]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [104]:
def GridSearch(model, params, df, cv):

    X = df[num_cols]
    y = df['cardio']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

    model_CV = GridSearchCV(model, params, cv=cv, scoring='accuracy', n_jobs=-1)
    model_CV.fit(X_train, y_train)
    best_estimator = model_CV.best_estimator_


    pred = best_estimator.predict(X_test)
    metrics = {'accuracy': accuracy_score(y_test, pred),
            'roc_auc': roc_auc_score(y_test, pred),
            'f1': f1_score(y_test, pred)} 


    return metrics

In [102]:
param_knn = {'n_neighbors': np.arange(1, 7, 1)}
param_tree = {'max_depth': np.arange(1, 10, 1)}

knn = KNeighborsClassifier()
tree = DecisionTreeClassifier()

In [105]:
knn_best = GridSearch(knn, param_knn, df, 5)
tree_best = GridSearch(tree, param_tree, df, 5)

print(f'knn: {knn_best}\ntree: {tree_best}')

knn: {'accuracy': 0.680047619047619, 'roc_auc': 0.680146059089396, 'f1': 0.6434219604097012}
tree: {'accuracy': 0.7162380952380952, 'roc_auc': 0.7162810578512996, 'f1': 0.7030447999202671}


**Комментарии:** Ваши комментарии здесь.